In [ ]:
import pandas as pd

file_name=['01','02','03','04','05','06','07','08','09','10',
           '11','12','13','14','15','16','17','18','19','20',
           '21','22','23','24','25','26','27','28','29','30',
           '31','32','33','34','35','36','37','38','39','40',
           '41','42','43','44','45','46','47','48','49','50',
           '51','52','53','54','55','56','57','58','59','60']
data=[]

for i in range(len(file_name)):
    data.append(pd.read_csv('../data/'+file_name[i]+'_tracks.csv'))

In [ ]:
#DV5
import numpy as np
for i in range(len(file_name)):
    length = len(data[i]['xAcceleration'][data[i]['xAcceleration']<0])
    dlong = data[i]['xAcceleration'][data[i]['xAcceleration']<0]
    dlong_mean_4_each_driver = data[i]['xAcceleration'][data[i]['xAcceleration']<0].groupby([data[i]['id']], dropna = False).mean()
    dlong_mean = data[i]['xAcceleration'][data[i]['xAcceleration']<0].mean()
    data[i]['DV5'] = np.sqrt((dlong - dlong_mean_4_each_driver)**2/length)/dlong_mean
    print(dlong)

In [ ]:
for i in range(len(file_name)):
    length = len(data[i]['xAcceleration'])
    Along = data[i]['xAcceleration']
    Along_mean = data[i]['xAcceleration'].mean()
    data[i]['DV7'] = (Along - Along_mean).abs()/length 
    print(data[i]['DV7'])